In [1]:
import numpy as np
from collections.abc import Iterable, Sequence, Callable
from scipy.optimize import brentq, fsolve
from tqdm import tqdm, trange

Определим функции и диапазоны параметров

In [2]:
ERROR_RATE = 0.05
N_DIM = 3
PARAM_DIM = 4
A_EXACT = np.array([2, 3, -1, 4])

In [3]:
def f_1(x: Sequence[float], a: Sequence[float]) -> float:
    return x[0]**2 * a[3] + x[1]**2 * a[0] + x[0] + x[2]**3 + x[1] * a[1] + a[2]

def f_1_grad(x: Sequence[float], a: Sequence[float]) -> np.ndarray[float]:
    return np.array([
        2 * x[0] * a[3] + 1,
        2 * x[1] * a[0],
        3 * x[2] * x[2]
    ])

def f_2(x: Sequence[float], a: Sequence[float]) -> float:
    return x[1]**2 + x[2]**2 + a[1] * x[1] * x[2] - a[0]
 
def f_2_grad(x: Sequence[float], a: Sequence[float]) -> np.ndarray[float]:
    return np.array([
        type(a[0])(0),
        2 * x[1] + x[2] * a[1],
        2 * x[2] + x[1] * a[1]
    ])

def f_3(x: Sequence[float], a: Sequence[float]) -> float:
    return a[0] * x[2]**2 + x[0]**2 + x[0] * x[2] - x[1] - a[3]

def f_3_grad(x: Sequence[float], a: Sequence[float]) -> np.ndarray[float]:
    return np.array([
        2 * x[0] + x[2],
        type(a[0])(-1),
        2 * x[2] * a[0] + x[0]
    ])

def F(x: Sequence[float], a: Sequence[float]) -> np.ndarray[float]:
    return np.array([f_1(x, a), f_2(x, a), f_3(x, a)])

def jacobi_F(x: Sequence[float], a: Sequence[float]) -> np.ndarray[float]:
    return np.array([f_1_grad(x, a), f_2_grad(x, a), f_3_grad(x, a)])

In [4]:
max_deltas = np.abs(A_EXACT * ERROR_RATE)
a_measured = A_EXACT + np.array([np.random.uniform(low=-delta, high=delta) for delta in max_deltas])
a_interval = list(zip(A_EXACT - max_deltas, A_EXACT + max_deltas))
a_interval

[(1.9, 2.1), (2.85, 3.15), (-1.05, -0.95), (3.8, 4.2)]

In [5]:
def derivatives_complex_step(func: Callable[[Sequence[float], Sequence[float]], float | Sequence[float]],
                             x_a: Sequence[float],
                             h: float = 1e-15
                             ) -> np.ndarray[float]:
    def f_wrapper(x_a: Sequence[float]):
        return func(x_a[:N_DIM], x_a[N_DIM:N_DIM+PARAM_DIM])
    x_a = np.array(x_a, dtype=complex)
    derivatives = [0.0] * len(x_a)
    for i in range(len(x_a)):
        x_a[i] += 1j * h
        derivatives[i] = f_wrapper(x_a).imag / h
        x_a[i] -= 1j * h
    return np.array(derivatives)

Аналитическая оценка погрешности корней векторнозначной функции $n$ переменных:

In [6]:
def analytical_ndim_roots_error_estimate(func: Callable[[Sequence[float], Sequence[float]], Sequence[float]],
                                         roots_estimates: Iterable[tuple],
                                         params: Sequence[float],
                                         params_max_error: Sequence[float],
                                         return_intervals: bool = False
                                         ) -> list[np.ndarray[float]]:
    res = []
    for root_estimate in roots_estimates:
        root = fsolve(func, root_estimate, args=(params,))
        derivatives = derivatives_complex_step(func, np.append(root, params)).T
        J_x = derivatives[:,:N_DIM]
        J_a = derivatives[:,N_DIM:]
        root_errors = np.abs(np.linalg.inv(J_x) @ J_a) @ params_max_error
        if return_intervals:
            res.append(list(zip(root - root_errors, root + root_errors)))
        else:
            res.append(root_errors)
    return res

Функция для каждого указанного корня возвращает список кортежей, которые являются границами значений соответствующих координат вектора корня, учитывая погрешности коэффициентов

In [7]:
analytical_ndim_roots_error_estimate(F, [(1, 0, -1), (-1, 0, -1)], a_measured, max_deltas, return_intervals=True)

[[(0.8954388073931758, 0.9659393411886912),
  (-0.0009565057920609549, 0.17324682226834726),
  (-1.6439626313463724, -1.444025613681147)],
 [(-0.9804060957850156, -0.9309455172822569),
  (-0.41949715717808833, -0.1876527127585541),
  (-1.077254817986466, -0.8855103221632901)]]

Точные значения корней на самом деле такие:

In [8]:
fsolve(F, (1, 0, -1), args=(a_measured,)), fsolve(F, (-1, 0, -1), args=(a_measured,))

(array([ 0.93068907,  0.08614516, -1.54399412]),
 array([-0.95567581, -0.30357493, -0.98138257]))

Далее нам придётся находить корни функций на интервалах (по одному на каждом интервале). Определим для удобства соответствующую функцию, применяющую к каждому интервалу солвер `brentq` из `scipy.optimize`.

Определим метод Монте-Карло нахождения погрешностей корней 

In [9]:
def monte_carlo_roots_error_estimate(func: Callable[[float, Sequence[float]], float],
                                     roots_estimates: Sequence[tuple],
                                     params_intervals: Sequence[tuple[float, float]],
                                     iterations: int = 10000
                                     ) -> list[list[tuple[float, float]]]:
    res = []
    for root_estimate in roots_estimates:
        root_values = []
        for _ in trange(iterations):
        # generate params
            cur_params = [np.random.uniform(*param_boundaries) for param_boundaries in params_intervals]
            root = fsolve(func, root_estimate, args=(cur_params,))
            root_values.append(root)
        res.append(list(zip(np.min(root_values, axis=0), np.max(root_values, axis=0))))
    return res

In [10]:
monte_carlo_roots_error_estimate(F, [(1, 0, -1), (-1, 0, -1)], a_interval, iterations=100_000)

100%|██████████| 100000/100000 [00:09<00:00, 10514.77it/s]


[[(0.8890625682357374, 0.955510847344109),
  (-0.017574149842581332, 0.16303942380421288),
  (-1.6212585018041061, -1.4239883148693961)],
 [(-0.978528397820306, -0.9264970741556366),
  (-0.4553670155609807, -0.22472583749479894),
  (-1.0536479204625964, -0.866426767818298)]]

# EXTRA

Я не закончил, но попытался (

In [11]:
import os, sys
scripts_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'practice')
if not scripts_dir in sys.path:
    sys.path.append(scripts_dir)
from utils import AffineArithmetics, FwdAAD, BwdAAD

In [12]:
import intvalpy as ip 

# Implementation based on
# https://www.researchgate.net/publication/282741908_Solving_multidimensional_nonlinear_perturbed_problems_using_interval_Newton_methods?enrichId=rgreq-0e1edeed36e091bdb9fadde0ab05c3e8-XXX&enrichSource=Y292ZXJQYWdlOzI4Mjc0MTkwODtBUzoxMDU0ODQ1Mjc2NTMyNzM2QDE2Mjg1MDYxMjAxMTQ%3D&el=1_x_3&_esc=publicationCoverPdf
def interval_newton_root_error_estimate(func: Callable[[float, Sequence[float]], Sequence[float]],
                                        jacobi: Callable[[float, Sequence[float]], np.ndarray],
                                        root_interval: list[tuple],
                                        params_intervals: Sequence[tuple[float, float]],
                                        iterations: int = 10):
    X_0 = [AffineArithmetics(*coord_interval) for coord_interval in root_interval]
    P = [AffineArithmetics(*a) for a in params_intervals]
    X_k = X_0
    for _ in range(iterations):
        J_x = jacobi(X_0, P)
        J_x_list = np.zeros((*J_x.shape, 2))
        for i in range(J_x.shape[0]):
            for j in range(J_x.shape[1]):
                J_x_list[i, j] = J_x[i, j].to_list()
        A = ip.Interval(J_x_list)
        m_X_k = [x.midpoint() for x in X_k]
        b = func(m_X_k, P)
        print(A)
        Z = ip.linear.Gauss_Seidel(A, b)
        Z = AffineArithmetics(Z.a, Z.b)
        interval_newton_operator = [m + z for m, z in zip(m_X_k, Z)]
        X_k = [x.intersect(n) for x, n in zip(X_k, interval_newton_operator)]
        print(X_k)

In [13]:
interval_newton_root_error_estimate(F, jacobi_F, root_interval=[(-1, 0), (-1, 0), (-2, -1)], params_intervals=a_interval)

Interval([['[-7.4, 1.4]', '[-4.2, 0.2]', '[1.5, 12]'],
       ['[0, 0]', '[-8.3, -2.7]', '[-7.15, -1.85]'],
       ['[-4, -1]', '[-1, -1]', '[-9.4, -3.6]']])


AssertionError: Matrix of the system not an H-matrix